In [1]:
global file
file = input('Selet a folder to save output') ##enter your file path here - the file is in the repo "summary_stats.csv".
raw_file = file

In [1]:
from datetime import datetime
from datetime import date
from subgrounds.subgraph import SyntheticField, FieldPath
from subgrounds.subgrounds import Subgrounds
import math
import pandas as pd
import  os as os
import duckdb as db
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
sg = Subgrounds()
holders = sg.load_api('https://api.studio.thegraph.com/query/28103/token-holders/0.0.13')

In [3]:
token_balances = holders.Query.tokenHolderBalances(
    orderBy='timestamp',
    first=1000 ,
    orderdirection = 'asc',
    #where=[
      #      token_balances.date  > '2022-09-24'
     #     ]
)

In [4]:
balances = sg.query_df([
    token_balances.date,
    token_balances.holder.token,
    token_balances.holder.holder,
    token_balances.balance
])
balances

,tokenHolderBalances_date,tokenHolderBalances_holder_token_id,tokenHolderBalances_holder_token_address,tokenHolderBalances_holder_token_blockchain,tokenHolderBalances_holder_token_name,tokenHolderBalances_holder_holder,tokenHolderBalances_balance
0,2021-11-24T08:18:17.000Z,gOHM/Ethereum,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Ethereum,gOHM,0xf9704b03e94b8c19cfd8a8803d81c95e814d2a44,5.358220e-04
1,2021-11-24T08:21:28.000Z,gOHM/Ethereum,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Ethereum,gOHM,0xf9704b03e94b8c19cfd8a8803d81c95e814d2a44,0.000000e+00
2,2021-11-24T08:22:37.000Z,gOHM/Ethereum,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Ethereum,gOHM,0xf9704b03e94b8c19cfd8a8803d81c95e814d2a44,5.368538e-04
3,2021-11-24T08:58:07.000Z,gOHM/Ethereum,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Ethereum,gOHM,0x5d8b27b839d86edf67a334139e8d51a5348eeba2,1.000000e-09
4,2021-11-24T08:58:07.000Z,gOHM/Ethereum,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Ethereum,gOHM,0xf9704b03e94b8c19cfd8a8803d81c95e814d2a44,5.368528e-04
...,...,...,...,...,...,...,...
995,2021-11-30T09:14:22.000Z,gOHM/Ethereum,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Ethereum,gOHM,0x131b4e746ca5a5a8be68d2bb5f5a8533a4dd99c1,4.249270e+01
996,2021-11-30T09:14:49.000Z,gOHM/Ethereum,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Ethereum,gOHM,0x2796317b0ff8538f253012862c06787adfb8ceb6,1.157304e+02
997,2021-11-30T09:14:49.000Z,gOHM/Ethereum,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Ethereum,gOHM,0x374632115489102b83f66ff0d45dc834949c0923,5.119263e-02
998,2021-11-30T09:14:49.000Z,gOHM/Ethereum,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Ethereum,gOHM,0x6571d6be3d8460cf5f7d6711cd9961860029d85f,0.000000e+00


token_balances_exp = holders.Query.tokenHolderBalances(
    orderBy='timestamp',
    orderdirection = 'asc',
    first=1000,
    skip= 1000,
    where=[
            token_balances.date  > '2022-05-05'
          ]
)
balances_exp = sg.query_df([
    token_balances_exp.date,
    token_balances_exp.holder.token,
    token_balances_exp.holder.holder,
    token_balances_exp.balance
])
balances_exp

In [5]:
##ok. We gotta retrieve pretty much every frickin' row. But only 1000 rows at a time, skipping by 1000 with every iteration. Until the number of rows returned is less than 1000. Orash
n=0
done = 0
rowcount = 0
while done == 0:
    n=n+1
    token_balances_2 = holders.Query.tokenHolderBalances(
        orderBy='timestamp',
        first=1000,
        skip = 1000*1+n,
        orderdirection = 'asc',
        #where=[
         #   token_balances.date  > '2022-09-24'
         # ]
    )

    balances_2 = sg.query_df([
        token_balances_2.date,
        token_balances_2.holder.token,
        token_balances_2.holder.holder,
        token_balances_2.balance
    ])
    last_date = max(balances['tokenHolderBalances_date'])
    rowcount = len(balances_2)
    print("iteration:",n,"rows:",rowcount, last_date)
    frames = [balances, balances_2]
    balances = pd.concat(frames)
    if rowcount<1000:
        done=1

balances


iteration: 1 rows: 1000 2021-11-30T09:14:49.000Z
iteration: 2 rows: 1000 2021-12-01T02:15:02.000Z
iteration: 3 rows: 1000 2021-12-01T02:17:31.000Z
iteration: 4 rows: 1000 2021-12-01T02:17:31.000Z
iteration: 5 rows: 1000 2021-12-01T02:32:20.000Z
iteration: 6 rows: 1000 2021-12-01T02:33:03.000Z
iteration: 7 rows: 1000 2021-12-01T02:33:54.000Z
iteration: 8 rows: 1000 2021-12-01T02:36:12.000Z
iteration: 9 rows: 1000 2021-12-01T02:36:12.000Z
iteration: 10 rows: 1000 2021-12-01T02:36:12.000Z
iteration: 11 rows: 1000 2021-12-01T02:40:58.000Z
iteration: 12 rows: 1000 2021-12-01T02:40:58.000Z
iteration: 13 rows: 1000 2021-12-01T02:40:58.000Z
iteration: 14 rows: 1000 2021-12-01T02:43:04.000Z
iteration: 15 rows: 1000 2021-12-01T02:43:04.000Z
iteration: 16 rows: 1000 2021-12-01T02:43:04.000Z
iteration: 17 rows: 1000 2021-12-01T02:44:04.000Z
iteration: 18 rows: 1000 2021-12-01T02:44:04.000Z
iteration: 19 rows: 1000 2021-12-01T02:46:40.000Z
iteration: 20 rows: 1000 2021-12-01T02:47:45.000Z
iteration

KeyboardInterrupt: 

In [63]:
balances.to_csv("balances.csv", index = False)